# Тестирование

[Levels of testing](https://softwaretestingfundamentals.com/software-testing-levels/):

- Unit Testing: 	A level of the software testing process where individual units of a software are tested. The purpose is to validate that each unit of the software performs as designed.

- Integration Testing: 	A level of the software testing process where individual units are combined and tested as a group. The purpose of this level of testing is to expose faults in the interaction between integrated units.

- System Testing: 	A level of the software testing process where a complete, integrated system is tested. The purpose of this test is to evaluate the system’s compliance with the specified requirements.

- Acceptance Testing: 	A level of the software testing process where a system is tested for acceptability. The purpose of this test is to evaluate the system’s compliance with the business requirements and assess whether it is acceptable for delivery.

Сегодня мы концентрируемся на первом уровне -- юнит тестах. И конкретно на стандартной имплементации с помощью модуля [unittest](https://docs.python.org/3/library/unittest.html)

The unittest unit testing framework was originally inspired by JUnit and has a similar flavor as major unit testing frameworks in other languages. It supports test automation, sharing of setup and shutdown code for tests, aggregation of tests into collections, and independence of the tests from the reporting framework.

To achieve this, unittest supports some important concepts in an object-oriented way:

* **test fixture**
A test fixture represents the preparation needed to perform one or more tests, and any associated cleanup actions. This may involve, for example, creating temporary or proxy databases, directories, or starting a server process.

* **test case**
A test case is the individual unit of testing. It checks for a specific response to a particular set of inputs. unittest provides a base class, TestCase, which may be used to create new test cases.

* **test suite**
A test suite is a collection of test cases, test suites, or both. It is used to aggregate tests that should be executed together.

* **test runner**
A test runner is a component which orchestrates the execution of tests and provides the outcome to the user. The runner may use a graphical interface, a textual interface, or return a special value to indicate the results of executing the tests.

## Doctest

In [53]:
%%writefile gaf.py

def gaf(length=1, end=''):
    '''
    Гавкнуть длиной length с end в конце
    Параметры необязательные

    >>> gaf()
    'Gaf'

    >>> gaf(3)
    'Gaaaf'

    >>> gaf(4, '!')
    'Gaaaaf!'

    '''


    return 'G' + 'a' * length + 'f' + end

Overwriting gaf.py


In [56]:
!python3 -m doctest -v gaf.py

Trying:
    gaf()
Expecting:
    'Gaf'
ok
Trying:
    gaf(3)
Expecting:
    'Gaaaf'
ok
Trying:
    gaf(4, '!')
Expecting:
    'Gaaaaf!'
ok
1 items had no tests:
    gaf
1 items passed all tests:
   3 tests in gaf.gaf
3 tests in 2 items.
3 passed and 0 failed.
Test passed.


К стати, -m загружает пакет.
https://sentry.io/answers/what-is-init-py-for-in-python/# - тут немного деталей по нотации файлов в пакете. Есть похожая на дексприторы нотация `__init__.py.`



## Unittest

### Тестируем отдельные функции

In [59]:
%%writefile calculations.py

def add(x: float, y: float) -> float:
    return x + y

def divide(x: float, y: float) -> float:

    try:
        res = x / y

    except ZeroDivisionError as e:
        raise ValueError(e)
        res = 0

    return res

Writing calculations.py


In [62]:
%%writefile test_calculations.py

import unittest

import calculations


class TestCase(unittest.TestCase):

    def test_add(self):

        test_cases = [(4, 4), (10, 15), (-10, 10), (0.1, 0.2)]
        test_answers = [8, 25, 0, 0.3]

        for inputs, answer in zip(test_cases, test_answers):
            result = calculations.add(*inputs)
            self.assertAlmostEqual(result, answer)
            self.assertEqual(result, answer)


    def test_divide(self):
        test_cases = [(10, 1), (10, 2), (5, 2)]
        test_answers = [10, 5, 2.5]

        for inputs, answer in zip(test_cases, test_answers):
            result = calculations.divide(*inputs)
            self.assertAlmostEqual(result, answer)

        with self.assertRaises(ValueError):
            res = calculations.divide(1, 0)

if __name__ == '__main__':
    unittest.main()

Overwriting test_calculations.py


In [63]:
! python3 -m unittest -v test_calculations.py

test_add (test_calculations.TestCase) ... FAIL
test_divide (test_calculations.TestCase) ... ok

FAIL: test_add (test_calculations.TestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/content/test_calculations.py", line 17, in test_add
    self.assertEqual(result, answer)
AssertionError: 0.30000000000000004 != 0.3

----------------------------------------------------------------------
Ran 2 tests in 0.001s

FAILED (failures=1)


### Тестируем класс

In [ ]:
%%writefile student.py

class Student:

    coeff_scholarship = 2

    def __init__(self, first_name, second_name, year, scholarship):
        self.first_name = first_name
        self.second_name = second_name
        self.year = year
        self.scholarship = scholarship

    @property
    def email(self):
        return f'{self.first_name.lower()[0]}{self.second_name.lower()}@edu.hse.ru'

    def increase_scholarship(self):
        self.scholarship = int(self.scholarship * self.coeff_scholarship)

Overwriting student.py


https://docs.python.org/3/library/functions.html#property @property documentation

In [ ]:
%%writefile test_student.py

import unittest

from student import Student

class TestStudent(unittest.TestCase):

    def setUp(self):
        '''
        Выполняется перед тестами
        '''

        self.default_name = 'Denis'
        self.default_surname = 'Belyakov'
        self.default_year = 2015
        self.default_scholar = 1

        self.t_st = Student(
            self.default_name,
            self.default_surname,
            self.default_year,
            self.default_scholar,
        )

    def tearDown(self):
        '''
        Выполняется, когда все тесты отработали
        '''

        pass

    def test_email(self):
        self.assertEqual(self.t_st.email, 'dbelyakov@edu.hse.ru')

        self.t_st.first_name = 'Kirill'
        self.assertEqual(self.t_st.email, 'kbelyakov@edu.hse.ru')

        self.t_st.first_name = self.default_name  # возвращаем атрибуты к дефолтным значениям

    def test_increase_scholarship(self):
        self.t_st.increase_scholarship()
        self.assertGreaterEqual(self.t_st.scholarship, self.default_scholar)

        self.t_st.scholarship = self.default_scholar


if __name__ == '__main__':
    unittest.main()

Overwriting test_student.py


In [ ]:
! python3 -m unittest -v test_student.py

test_email (test_student.TestStudent) ... ok
test_increase_scholarship (test_student.TestStudent) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


# Как работают unittest.mock: patch vs Mock
https://kirill-sklyarenko.ru/lenta/python-patches-mocks-anti-patterns неплохая статья о том, как осознанно использовать конструкты тестирования


patch можно так же использовать как аннтотацию, тогда сгенерированный мок передается как параметр функции

In [66]:
%%writefile mock_db.py

from unittest.mock import patch

def db_read(item_id):
    return None # просто для примера

def total_value(item_id):
    items = db_read(item_id)
    return sum(items)

Overwriting mock_db.py


In [76]:
%%writefile test_db.py

from unittest.mock import patch
from mock_db import total_value

import unittest

class TestDb(unittest.TestCase):
    def test_total_value(self):
        with patch('mock_db.db_read') as db_read:
            db_read.return_value = [100, 200]  # это у нас заглушка
            assert 300 == total_value(1234)
            db_read.assert_called_with(1234) # а это верификация

Overwriting test_db.py


In [79]:
%%writefile test_db.py

from unittest.mock import patch
from mock_db import total_value

import unittest

class TestDb(unittest.TestCase):
  @patch('mock_db.db_read')
  def test_total_value(self, mock_db_read):
    mock_db_read.return_value = [100, 200]

    assert 300 == total_value(1234)

    mock_db_read.assert_called_with(1234)


Overwriting test_db.py


In [85]:
%%writefile test_db.py

from unittest.mock import Mock
from mock_db import total_value

import unittest

class TestDb(unittest.TestCase):

  def test_total_value(self):
    mock_db_read = Mock()
    mock_db_read.return_value = [100, 200]

    total_value.__globals__['db_read'] = mock_db_read

    assert 300 == total_value(1234)

    mock_db_read.assert_called_with(1234)


Overwriting test_db.py


In [86]:
! python3 -m unittest -v test_db.py

test_total_value (test_db.TestDb) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


### Тестируем TelegramBot с прошлого семинара

Возьмем класс бота с прошлого занятия

In [38]:
%%writefile telegram_bot.py

import os
from groq import Groq
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import ApplicationBuilder, CommandHandler, ContextTypes, MessageHandler, filters, CallbackQueryHandler


class TelegramBot:
    def __init__(self):
        self.client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
        self.app = ApplicationBuilder().token(os.getenv('TELEGRAM_KEY')).build()
        self.app.add_handler(CommandHandler("start", self.start))
        self.app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, self.handle_message))
        self.app.add_handler(MessageHandler(filters.COMMAND, self.unknown_command))
        self.app.add_handler(CallbackQueryHandler(self.button_handler))

    async def start(self, update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
        """Handle the /start command and send a welcome message with an inline button."""
        keyboard = [
            [InlineKeyboardButton("About us", callback_data='creator_info')]
        ]
        reply_markup = InlineKeyboardMarkup(keyboard)

        await update.message.reply_text(f'Hello {update.effective_user.first_name}', reply_markup=reply_markup)

    async def button_handler(self, update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
        """Handle button interactions from the inline keyboard."""
        query = update.callback_query
        await query.answer()

        if query.data == 'creator_info':
            await query.edit_message_text(text="We are cool people.")

    async def handle_message(self, update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
        """Handle any text message sent by the user."""
        user_message = update.message.text
        response = self.send_to_groq_api(user_message)
        await update.message.reply_text(response)

    def send_to_groq_api(self, message: str) -> str:
        """Send a message to the Groq API and return the response."""
        try:
            print(f"Sending message to Groq API: {message}")
            chat_completion = self.client.chat.completions.create(
                messages=[
                    {
                        "role": "user",
                        "content": message,
                    }
                ],
                model="mixtral-8x7b-32768",
            )
            print(f"Received response from Groq API: {chat_completion.choices[0].message.content}")
            return chat_completion.choices[0].message.content

        except Exception as e:
            return f"An error occurred while communicating with the Groq API: {e}"

    async def unknown_command(self, update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
        """Handle unknown commands."""
        await update.message.reply_text("Sorry, I didn't understand that command.")

    def run(self):
        """Start the bot and begin polling for updates."""
        self.app.run_polling()


if __name__ == "__main__":
    bot = TelegramBot()
    bot.run()

Overwriting telegram_bot.py


Функция похода по api зависит от того, как работает Groq, а мы хотим проверять именно наш код. Можно использовать механизм "моков" и определить ("замокать") какие-то значения для функции, как будто мы ее правда вызвали.

In [23]:
%pip install python-telegram-bot groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 2.9 MB/s eta 0:00:00


In [46]:
%%writefile test_telegram_bot.py

import unittest
from unittest.mock import patch, AsyncMock, MagicMock
from telegram import Update, InlineKeyboardMarkup, InlineKeyboardButton
import telegram_bot


class TestTelegramBot(unittest.IsolatedAsyncioTestCase):

    @patch('telegram_bot.Groq')  # Mock Groq client
    @patch('telegram_bot.ApplicationBuilder')  # Mock Telegram ApplicationBuilder
    def setUp(self, mock_app_builder, mock_groq):
        self.mock_groq = mock_groq
        self.mock_app_builder = mock_app_builder
        self.bot = telegram_bot.TelegramBot()  # Create bot instance with mocked dependencies

    @patch('telegram_bot.Update')
    async def test_start(self, mock_update):
        """Test the /start command."""
        mock_update.message = AsyncMock()
        mock_update.effective_user.first_name = 'TestUser'

        # Call the /start command handler
        await self.bot.start(mock_update, None)

        # Ensure the welcome message with inline buttons is sent
        mock_update.message.reply_text.assert_called_once_with(
            'Hello TestUser',
            reply_markup=InlineKeyboardMarkup([[InlineKeyboardButton("About us", callback_data='creator_info')]])
        )

    @patch('telegram_bot.Update')
    async def test_button_handler(self, mock_update):
        """Test the button interaction."""
        mock_update.callback_query = AsyncMock()
        mock_update.callback_query.data = 'creator_info'

        # Call the button handler
        await self.bot.button_handler(mock_update, None)

        # Ensure the query is answered and the message is updated
        mock_update.callback_query.answer.assert_called_once()
        mock_update.callback_query.edit_message_text.assert_called_once_with(text="We are cool people.")

    @patch('telegram_bot.Update')
    @patch.object(telegram_bot.TelegramBot, 'send_to_groq_api')
    async def test_handle_message(self, mock_send_to_groq_api, mock_update):
        """Test handling of a text message."""
        mock_update.message = AsyncMock()
        mock_update.message.text = "Hello bot"
        mock_send_to_groq_api.return_value = "Hello user"

        # Call the message handler
        await self.bot.handle_message(mock_update, None)

        # Ensure the message was sent to the Groq API and the response was sent back to the user
        mock_send_to_groq_api.assert_called_once_with("Hello bot")
        mock_update.message.reply_text.assert_called_once_with("Hello user")

    @patch('telegram_bot.Update')
    async def test_unknown_command(self, mock_update):
        """Test handling of unknown commands."""
        mock_update.message = AsyncMock()

        # Call the unknown command handler
        await self.bot.unknown_command(mock_update, None)

        # Ensure an appropriate reply is sent for unknown commands
        mock_update.message.reply_text.assert_called_once_with("Sorry, I didn't understand that command.")

if __name__ == "__main__":
    unittest.main()


Overwriting test_telegram_bot.py


In [47]:
! python3 -m unittest -v test_telegram_bot.py

test_button_handler (test_telegram_bot.TestTelegramBot)
Test the button interaction. ... ok
test_handle_message (test_telegram_bot.TestTelegramBot)
Test handling of a text message. ... ok
test_start (test_telegram_bot.TestTelegramBot)
Test the /start command. ... ok
test_unknown_command (test_telegram_bot.TestTelegramBot)
Test handling of unknown commands. ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.045s

OK


# А как прорефакторить класс, что бы он был легче тестирумемым и удовлетворял принципам SOLID?

In [ ]:
import os
from groq import Groq
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import ApplicationBuilder, CommandHandler, ContextTypes, MessageHandler, filters, CallbackQueryHandler

class GroqClient:
    """Handles communication with the Groq API."""
    def __init__(self, client=None):
        self.client = client or Groq(api_key=os.getenv("GROQ_API_KEY"))

    def send_message(self, message: str) -> str:
        """Send a message to the Groq API and return the response."""
        try:
            print(f"Sending message to Groq API: {message}")
            chat_completion = self.client.chat.completions.create(
                messages=[
                    {
                        "role": "user",
                        "content": message,
                    }
                ],
                model="mixtral-8x7b-32768",
            )
            return chat_completion.choices[0].message.content

        except Exception as e:
            return f"An error occurred while communicating with the Groq API: {e}"


class TelegramBot:
    """Handles Telegram bot commands and interactions."""
    def __init__(self, api_client: GroqClient):
        self.client = api_client
        self.app = ApplicationBuilder().token(os.getenv('TELEGRAM_KEY')).build()
        self.add_handlers()

    def add_handlers(self):
        """Add command and message handlers to the bot."""
        self.app.add_handler(CommandHandler("start", self.start))
        self.app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, self.handle_message))
        self.app.add_handler(MessageHandler(filters.COMMAND, self.unknown_command))
        self.app.add_handler(CallbackQueryHandler(self.button_handler))

    async def start(self, update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
        """Handle the /start command and send a welcome message with an inline button."""
        keyboard = [
            [InlineKeyboardButton("About us", callback_data='creator_info')]
        ]
        reply_markup = InlineKeyboardMarkup(keyboard)
        await update.message.reply_text(f'Hello {update.effective_user.first_name}', reply_markup=reply_markup)

    async def button_handler(self, update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
        """Handle button interactions from the inline keyboard."""
        query = update.callback_query
        await query.answer()

        if query.data == 'creator_info':
            await query.edit_message_text(text="We are cool people.")

    async def handle_message(self, update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
        """Handle any text message sent by the user."""
        user_message = update.message.text
        response = self.client.send_message(user_message)
        await update.message.reply_text(response)

    async def unknown_command(self, update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
        """Handle unknown commands."""
        await update.message.reply_text("Sorry, I didn't understand that command.")

    def run(self):
        """Start the bot and begin polling for updates."""
        self.app.run_polling()


if __name__ == "__main__":
    groq_client = GroqClient()
    bot = TelegramBot(api_client=groq_client)
    bot.run()


# А как задеплоить наш бот на сервер? - попробуем разобраться как работает ansible

нам нужен файл с доступами, playbook и командная строка с установленным клиентом, и ssh на машине сервере

In [ ]:
# hosts.ini

[telegram-bot]
<your_vm_ip> ansible_user=<your_vm_user< ansible_password=<your_vm_password>

In [ ]:
# deploy_telegram_bot.yml

---
- name: Deploy Telegram Bot to Ubuntu VM
  hosts: telegram-bot
  become: yes
  vars:
    bot_directory: /opt/telegram_bot
    virtualenv_directory: "{{ bot_directory }}/venv"
    telegram_api_key: "<your_telegram_api_key>"
    groq_api_key: "<your_groq_api_key>"

  tasks:
    - name: Ensure the required system packages are installed
      apt:
        name:
          - python3
          - python3-venv
          - python3-pip
          - git
        state: present
        update_cache: yes

    - name: Create bot directory
      file:
        path: "{{ bot_directory }}"
        state: directory
        owner: "{{ ansible_user }}"
        group: "{{ ansible_user }}"
        mode: '0755'

    - name: Copy the Telegram bot code to the VM
      copy:
        src: ./telegram_bot.py
        dest: "{{ bot_directory }}/telegram_bot.py"
        mode: '0755'

    - name: Copy the requirements.txt file (if needed)
      copy:
        src: ./requirements.txt
        dest: "{{ bot_directory }}/requirements.txt"
        mode: '0644'

    - name: Create virtual environment
      command: python3 -m venv "{{ virtualenv_directory }}"
      args:
        creates: "{{ virtualenv_directory }}/bin/activate"

    - name: Install dependencies in virtual environment
      command: "{{ virtualenv_directory }}/bin/pip install -r requirements.txt"
      args:
        chdir: "{{ bot_directory }}"

    - name: Create environment variables file
      copy:
        content: |
          export TELEGRAM_KEY="{{ telegram_api_key }}"
          export GROQ_API_KEY="{{ groq_api_key }}"
        dest: "{{ bot_directory }}/.env"
        mode: '0644'

    - name: Create systemd service file for the Telegram bot
      copy:
        dest: /etc/systemd/system/telegram_bot.service
        content: |
          [Unit]
          Description=Telegram Bot Service
          After=network.target

          [Service]
          User={{ ansible_user }}
          WorkingDirectory={{ bot_directory }}
          EnvironmentFile={{ bot_directory }}/.env
          ExecStart={{ virtualenv_directory }}/bin/python {{ bot_directory }}/telegram_bot.py
          Restart=always

          [Install]
          WantedBy=multi-user.target
        mode: '0644'

    - name: Reload systemd to apply the new service
      systemd:
        daemon_reload: yes

    - name: Enable and start the Telegram bot service
      systemd:
        name: telegram_bot.service
        enabled: yes
        state: started



Вообще jenkins gitlab CICD работаю похожим образом, есть скрипт, который разделен на этапы и каждый этап выполняется в конмандной строке, зачастую предоставляются какие то красивые декларативные ручки, но по сути все это обычная командная строка на сервере

In [ ]:
# После того как вы на своей локальной машине установили клиент, вы можете запустить скрипт - локальной машиной так же может быть какой-то воркер или github actions
# Мир DevOps полезен и многообразен

ansible-playbook -i hosts.ini deploy_telegram_bot.yml